<a href="https://colab.research.google.com/github/Sep-eg/kaggle_survey2021/blob/main/2021_kaggle_survey_GHK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 데이터 불러오기

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle
!kaggle competitions download -c kaggle-survey-2021

In [ ]:
import pandas as pd

In [ ]:
response = pd.read_csv("/content/kaggle_survey_2021_responses.csv.zip")

In [ ]:
response.head()

In [ ]:
response = response.loc[:][1:]
response.reset_index(drop=True, inplace=True)

In [ ]:
response.head()

## 질문합치기
모두를 대상으로 질문한 총 42개의 메인 문항과

비전문가 계열에게 질문한 8개의 보충 문항을 분류한뒤 각각을 질문 카테고리별로 합칩니다.

In [ ]:
questions = response.columns

In [ ]:
supple_survey_columns = []
for quest in questions:
    if '_' in quest:
        if quest.split('_')[1] == 'B':
            supple_survey_columns.append(quest)
    else:
        pass

In [ ]:
main_survey_columns = questions.drop(supple_survey_columns)

In [ ]:
supple_dict = {col.split('_')[0] : [] for col in supple_survey_columns}
for col in supple_survey_columns:
    supple_dict[col.split('_')[0]].append(col)

main_dict = {col.split('_')[0]: [] for col in main_survey_columns}
for col in main_survey_columns:
    main_dict[col.split('_')[0]].append(col)

In [ ]:
supple_response = response[supple_survey_columns].fillna('Not responded')
main_response = response[main_survey_columns].fillna('Not responded')

In [ ]:
def split_dropnone(col): 
    tmp_list = [data for data in col.split(',') if data not in 'Not responded']
    if len(tmp_list) == 0:
        return 'Not responded'
    else:
        return tmp_list

In [ ]:
for key, value in supple_dict.items():
    tmp_series = pd.Series(supple_response[value[0]])
    for col in value[1:]:
        tmp_series = tmp_series + ',' + supple_response[col]
    supple_response[key] = tmp_series
    supple_response.drop(value, axis=1, inplace=True)
    supple_response[key] = supple_response[key].map(split_dropnone)

In [ ]:
for key, value in main_dict.items():
    tmp_series = pd.Series(main_response[value[0]])
    flag = 0
    if len(value) > 1:
        flag += 1
        for col in value[1:]:
            tmp_series = tmp_series + ',' + main_response[col]
        main_response[key] = tmp_series
    if flag == 1:
        main_response.drop(value, axis=1, inplace=True)
        main_response[key] = main_response[key].map(split_dropnone)

In [ ]:
main_response